In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import label_binarize
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import MultiLabelBinarizer
import numpy.testing as test
import warnings 

warnings.filterwarnings("ignore", category=FutureWarning) 

%matplotlib inline


class DataFrameSelector(BaseEstimator, TransformerMixin) :    
    def __init__(self, columns):
        self.columns=columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.columns].values

In [2]:
class MultiFeatureEncoder (MultiLabelBinarizer):
    def fit(self, X, y=None):
        return super().fit(X)
    def transform(self, X):
        return X[self.columns].values
    def fit_transform(self, X, y=None):
        return super().fit_transform(X.astype(str))

In [3]:

class NameEncoder(BaseEstimator, TransformerMixin) :    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        flop=['Master', 'Miss', 'Mrs.', 'Mr.']
        f_b = np.zeros(X.size, dtype=bool) # initialize as false
        for f in flop:
            f1 = (np.char.find(X[:,0].astype(str), f)!=-1) # which entries contain the f key
            X[f1]=flop.index(f) # replace with the actual index in the array string            
            f_b = f_b | f1 # retain the matched ones
        X[~f_b]=4 # undefined
        return X.astype(np.int)
    @staticmethod
    def test():
        nameEncoder = NameEncoder()
        res = nameEncoder.fit_transform(np.array([['Master Jimmy'], ['Mrs. Smith'], ['Mr. Smith']]))
        test.assert_array_equal(res, [[0],[2],[3]])


In [4]:
# age imputer expects an 2D array having on the first column
# the encoded name 0,1,2,3,4 and on the second column the age
class AgeImputer(BaseEstimator, TransformerMixin) :        
    # fit will calculate the averages per encoded name
    def fit(self, X, y=None):
        self.titleAgeDictionary = {}
        for key in np.arange(0, 5):
            try: # just to make sure for unit tests add a try/catch
                self.titleAgeDictionary[key] = np.nanmedian(X[X[:,0] == key][1])
            except:
                pass
        return self

    def transform(self, X, y=None):        
        for key in self.titleAgeDictionary.keys():
            X[(X[:,0] == key) & (np.isnan(X[:,1])),1]=self.titleAgeDictionary[key]
        return X

    def test(self):
        X=np.array([[1,12],
          [2,12],
          [1, np.NaN]])
        res = self.fit_transform(X)


In [72]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler, Normalizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression;

# read the dataframe
df = pd.read_csv('train.csv')

def hasCabin(x):
    return np.apply_along_axis(lambda t : 0 if ( t != t) else 1 ,arr=x ,axis=1).reshape(-1,1)
        
# print(df[df['Survived'] == 0])

pipeline = make_column_transformer(
    (['Name', 'Age'], make_pipeline(
                        make_union(
                            make_pipeline(DataFrameSelector(['Name']), NameEncoder()),
                            make_pipeline(DataFrameSelector(['Age']), StandardScaler()) ), # union
                       AgeImputer(), make_column_transformer(([0], OneHotEncoder()), remainder='passthrough' )) # pipeline age
    ), #tuple
    (['Sex', 'Pclass'], OneHotEncoder(sparse=False)),
    (['Embarked'], make_pipeline( SimpleImputer( strategy='constant', fill_value='U' ), OneHotEncoder(sparse=False))), 
    (['Fare'], make_pipeline( SimpleImputer(strategy='mean'), FunctionTransformer( lambda x : np.log2(x, where=x>0)), StandardScaler() )),
    (['Cabin'], make_pipeline(FunctionTransformer(hasCabin, validate=False), OneHotEncoder()) ), 
    (['SibSp', 'Parch'], make_pipeline(DataFrameSelector(['SibSp','Parch']), 
                                       FunctionTransformer(lambda x : (x[:,0] + x[:,1] ).reshape(-1,1)))), 
    (['Age'], make_pipeline( FunctionTransformer(lambda x : 1 if x < 18 else 0, validate=False), OneHotEncoder(sparse=False)) ), 
    (['SibSp', 'Parch'], 'passthrough'),
    (['PassengerId', 'Survived'], 'drop'),
    )
x = pipeline.fit_transform(df)

print(x.shape)

y = df['Survived'].values

lr =LogisticRegression();
# s = cross_val_score(lr, x, y, scoring='neg_mean_squared_error', cv=4)
# scores=np.log(-s)
# print(scores)
# print('Mean=', scores.mean())
# print('Standard Dev=', scores.std())

# check by getting 80/20
num_train=int(x.shape[0]*0.7)
x_train=x[:num_train,:]
y_train=y[:num_train]

x_test=x[num_train:,:]
y_test=y[num_train:]

lr.fit(x_train, y_train)

y_predict = lr.predict(x_test)

# print(y_predict[:20])
# print(y_test[:20])

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
confusion_matrix(y_test, y_predict)
prec_score = precision_score(y_test, y_predict)
rec_score = recall_score(y_test, y_predict)
f1_sc = f1_score(y_test, y_predict)
print("Precision Score=", prec_score)
print("Recall Score=", rec_score)
print("F1 Score=", f1_sc)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [73]:
import re
import collections

#d = df['Name'].apply(lambda x: re.split(r"['\ \.']+", re.split(r"[',']+", x)[1]) )
#d[1]



#s='Hopa,Mrs.. Mitica'

#re.split(r"['\ \.']+", re.split(r"[',']+", s)[1])[0]

pipeline = make_pipeline(DataFrameSelector(['Age']), FunctionTransformer(lambda x : 1 if x < 18 else 0, validate=False))
x = pipeline.fit_transform( df )



C:\anacoonda\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()